**Nome:** Thaís de Almeida Machado

### Pacotes e métodos

In [71]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pydotplus
from IPython.display import Image
from sklearn import datasets, tree

### Base de dados

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
data = pd.read_excel("/content/drive/My Drive/Atividades/Atividade 3 - Bases.xlsx")

In [13]:
data.head()

,ID_usuário,Gênero,Idade,Estado_civil,Atividade_no_site,Pesquisou_eletronicos_12m,Comprou_eletronicos_12m,Pesquisou_mídia_digital_18m,Comprou_mídia_digital_18m,Forma_pagamento,Adoção_eReader
0,9552,M,61,C,Esporádico,Sim,Sim,Sim,Não,Transferência,Late Majority
1,6757,M,27,C,Intermitente,Sim,Não,Sim,Não,Transferência,Innovator
2,3599,F,29,C,Esporádico,Sim,Sim,Sim,Sim,Transferência,Early Adopter
3,6811,M,49,C,Esporádico,Sim,Sim,Sim,Sim,Website,Early Adopter
4,4104,M,29,S,Esporádico,Sim,Sim,Sim,Não,Website,Late Majority


### Análise descritiva dos dados

In [14]:
# Verificando se há valores ausentes na base de dados:
data.isna().sum()

ID_usuário                     0
Gênero                         0
Idade                          0
Estado_civil                   0
Atividade_no_site              0
Pesquisou_eletronicos_12m      0
Comprou_eletronicos_12m        0
Pesquisou_mídia_digital_18m    0
Comprou_mídia_digital_18m      0
Forma_pagamento                0
Adoção_eReader                 0
dtype: int64

Não há valores ausentes na base de dados.

#### Variáveis numéricas

In [15]:
# Analisando a variável Idade
data['Idade'].describe().round(2)

count    661.00
mean      42.79
std       13.86
min       16.00
25%       32.00
50%       44.00
75%       55.00
max       66.00
Name: Idade, dtype: float64

#### Variáveis categóricas

In [45]:
data['Gênero'].value_counts()

M    346
F    315
Name: Gênero, dtype: int64

In [46]:
data['Estado_civil'].value_counts()

C    381
S    280
Name: Estado_civil, dtype: int64

In [47]:
data['Atividade_no_site'].value_counts()

Esporádico      424
Intermitente    183
Frequênte        54
Name: Atividade_no_site, dtype: int64

In [48]:
data['Pesquisou_eletronicos_12m'].value_counts()

Sim    613
Não     48
Name: Pesquisou_eletronicos_12m, dtype: int64

In [49]:
data['Comprou_eletronicos_12m'].value_counts()

Sim    339
Não    322
Name: Comprou_eletronicos_12m, dtype: int64

In [50]:
data['Pesquisou_mídia_digital_18m'].value_counts()

Sim    525
Não    136
Name: Pesquisou_mídia_digital_18m, dtype: int64

In [51]:
data['Comprou_mídia_digital_18m'].value_counts()

Não    364
Sim    297
Name: Comprou_mídia_digital_18m, dtype: int64

In [52]:
data['Forma_pagamento'].value_counts()

Website              235
Transferência        229
Cartão de crédito    104
Boleto bancário       93
Name: Forma_pagamento, dtype: int64

### Pré-processamento

In [55]:
# Removendo a variável de identificação do usuário pois não é relevante para o modelo
data_new = data.drop('ID_usuário', axis = 1)
data_new.head()

,Gênero,Idade,Estado_civil,Atividade_no_site,Pesquisou_eletronicos_12m,Comprou_eletronicos_12m,Pesquisou_mídia_digital_18m,Comprou_mídia_digital_18m,Forma_pagamento,Adoção_eReader
0,M,61,C,Esporádico,Sim,Sim,Sim,Não,Transferência,Late Majority
1,M,27,C,Intermitente,Sim,Não,Sim,Não,Transferência,Innovator
2,F,29,C,Esporádico,Sim,Sim,Sim,Sim,Transferência,Early Adopter
3,M,49,C,Esporádico,Sim,Sim,Sim,Sim,Website,Early Adopter
4,M,29,S,Esporádico,Sim,Sim,Sim,Não,Website,Late Majority


In [56]:
# Convertendo as variáveis categóricas em variáveis dummies
data_encoded = pd.get_dummies(data_new,
                              columns = ['Gênero', 'Estado_civil', 'Atividade_no_site', 'Pesquisou_eletronicos_12m', 'Comprou_eletronicos_12m', 'Pesquisou_mídia_digital_18m', 'Comprou_mídia_digital_18m', 'Forma_pagamento'],
                              drop_first=True) # argumento para evitar redundâncias (2 variáveis perfeitamente correlacionadas)
data_encoded.head()

,Idade,Adoção_eReader,Gênero_M,Estado_civil_S,Atividade_no_site_Frequênte,Atividade_no_site_Intermitente,Pesquisou_eletronicos_12m_Sim,Comprou_eletronicos_12m_Sim,Pesquisou_mídia_digital_18m_Sim,Comprou_mídia_digital_18m_Sim,Forma_pagamento_Cartão de crédito,Forma_pagamento_Transferência,Forma_pagamento_Website
0,61,Late Majority,1,0,0,0,1,1,1,0,0,1,0
1,27,Innovator,1,0,0,1,1,0,1,0,0,1,0
2,29,Early Adopter,0,0,0,0,1,1,1,1,0,1,0
3,49,Early Adopter,1,0,0,0,1,1,1,1,0,0,1
4,29,Late Majority,1,1,0,0,1,1,1,0,0,0,1


### Divisão treino e teste

A divisão será feita da seguinte forma: 80% dos dados para treino e 20% dos dados para teste.


In [57]:
x = data_encoded.loc[:, data_encoded.columns != 'Adoção_eReader']
y = data_encoded['Adoção_eReader']

In [58]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

### Modelagem - árvore de decisão

In [88]:
# Selecionando diversos conjuntos de hiperparâmetros para testar qual será o melhor
tree_params = {"criterion": ["gini", "entropy"],
               "max_depth": np.arange(3, 20),
               "min_samples_split": np.arange(2, 5),
               "min_samples_leaf": np.arange(2, 5),
               "max_leaf_nodes": np.array([2, 5, 10, 20, 50, 100]),
               "ccp_alpha": np.array([0, 0.1, 0.2, 0.5, 0.7, 1])
               }

tree_params

{'criterion': ['gini', 'entropy'],
 'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 'min_samples_split': array([2, 3, 4]),
 'min_samples_leaf': array([2, 3, 4]),
 'max_leaf_nodes': array([  2,   5,  10,  20,  50, 100]),
 'ccp_alpha': array([0. , 0.1, 0.2, 0.5, 0.7, 1. ])}

In [89]:
# Classificador da árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=2)

In [90]:
# Método GridSearchCV para procurar os melhores hiperparâmetros
grid_search = GridSearchCV(decision_tree, tree_params, cv=5, scoring='accuracy')

In [91]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2),
             param_grid={'ccp_alpha': array([0. , 0.1, 0.2, 0.5, 0.7, 1. ]),
                         'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'max_leaf_nodes': array([  2,   5,  10,  20,  50, 100]),
                         'min_samples_leaf': array([2, 3, 4]),
                         'min_samples_split': array([2, 3, 4])},
             scoring='accuracy')

In [92]:
# Melhores hiperparâmetros
best_params = grid_search.best_params_
best_params

{'ccp_alpha': 0.0,
 'criterion': 'gini',
 'max_depth': 11,
 'max_leaf_nodes': 100,
 'min_samples_leaf': 4,
 'min_samples_split': 2}

In [93]:
# Melhor árvore
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=11, max_leaf_nodes=100, min_samples_leaf=4,
                       random_state=2)

In [94]:
print("Acurácia na base de treino:", round(best_model.score(x_train, y_train), 2))

Acurácia na base de treino: 0.77


In [95]:
y_pred = grid_search.best_estimator_.predict(x_test)
print("Acurácia na base de teste:", round(accuracy_score(y_test, y_pred), 2))

Acurácia na base de teste: 0.57


In [96]:
# Medidas avaliativas nos dados de teste
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

 Early Adopter       0.53      0.58      0.56        43
Early Majority       0.50      0.47      0.49        38
     Innovator       0.50      0.57      0.53        21
 Late Majority       0.81      0.68      0.74        31

      accuracy                           0.57       133
     macro avg       0.58      0.58      0.58       133
  weighted avg       0.58      0.57      0.57       133



In [80]:
# Matriz de confusão dos dados de teste
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix, index=["Early Adopter", "Early Majority", "Innovator", "Late Majority"], columns=["Early Adopter (predicted)", "Early Majority (predicted)", "Innovator (predicted)", "Late Majority (predicted)"])
cnf_table

,Early Adopter (predicted),Early Majority (predicted),Innovator (predicted),Late Majority (predicted)
Early Adopter,25,9,8,1
Early Majority,14,18,3,3
Innovator,6,2,12,1
Late Majority,2,7,1,21


#### Visualização da árvore de decisão

In [87]:
# Árvore com os melhores hiperparâmetros
best_tree = DecisionTreeClassifier(max_depth=11, max_leaf_nodes=100, min_samples_leaf=4, random_state=2)
final_tree = best_tree.fit(x_train, y_train)

dot_data = tree.export_graphviz(final_tree,
                                out_file=None,
                                proportion=False,
                                rounded =True,
                                filled=True,
                                feature_names=np.arange(0,12),
                                class_names=["Early Adopter", "Early Majority", "Innovator", "Late Majority"])

# Desenhando o gráfico
graph = pydotplus.graph_from_dot_data(dot_data)

# Gráfico
Image(graph.create_png())

Output hidden; open in https://colab.research.google.com to view.

**Conclusão**

Para ajuste do modelo de árvore de decisão, foram testados diversos conjuntos de hiperparâmetros diferentes. A melhor árvore construída com base na acurácia obteve uma acurácia de 0,77 em treino e 0,57 em teste. Isso é um indicativo de que houve overfitting.

Olhando para as classes individualmente nos dados de teste, podemos observar que o modelo obteve uma precisão e um recall mais altos na classe "late majority", o que indica uma maior facilidade do modelo em prever esses casos. Em contrapartida, parece que o modelo "confundiu" com mais facilidade as demais classes de compradores, o que pode ser comprovado pela matriz de confusão.

Possíveis soluções para esse problema podem ser agrupar 2 classes em 1, ficando apenas com 2 classes no final (se for de interesse para o negócio), ou treinando outro modelo de machine learning nos dados e comparando os resultados.